# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [1]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
import os

%cd ..

from CNN_small_architecture import CNNSmall
from CNN_layers import conv_homemade
from CNN_layers import maxpool_homemade
from CNN_layers import batchnorm_homemade
from CNN_layers import linear_layer_homemade
from CNN_layers import elu_homemade
from CNN_layers import relu_homemade as ReLU
from Tests.helper_functions import tokenize, transform_input, compare, create_conv_homemade, \
                                create_batchnorm_homemade, create_maxpool_homemade

/home/emil/anaconda3/envs/dl/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


/home/emil/Documents/bachelor/bachelorprojekt/Python


In [2]:
MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 2
test_loader = DataLoader(test_set, batch_size=batch_size)

In [3]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [4]:
input_original_test, label_test = next(iter(test_loader))
input_homemade_test = transform_input(input_batch=input_original_test)

In [5]:
elu_homemade = elu_homemade.ELU()
relu_homemade = ReLU.relu()
conv1_homemade = create_conv_homemade(model_conv= model_original.conv1)
batchnorm1_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm1)
maxpool1_homemade = create_maxpool_homemade(model_maxpool= model_original.maxPool1)
conv2_homemade = create_conv_homemade(model_conv= model_original.conv2)
batchnorm2_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm2)
maxpool2_homemade = create_maxpool_homemade(model_original.maxPool2)
linear_homemade = linear_layer_homemade.linear_layer(model_original.lin.weight,model_original.lin.bias,2)

STRIDE ER PRÆDEFINERET
STRIDE ER PRÆDEFINERET


## Shape of the network architecture

In [10]:
print(model_original.batchNorm1.running_mean)

tensor([ 0.3605, -0.2362,  0.1336])


## Testing the first convolutional layer

In [ ]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# original conv1 filter on test
out_original = model_original.conv1(input_original_test)

print("Convolutional layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the first Batchnorm layer

In [ ]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm1_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm1(out_original)

print("BatchNorm layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the first ReLU layer

In [ ]:
out_homemade = relu_homemade(out_homemade)

out_original = model_original.relu1(out_original)

print("ReLU layer 1 error over out channels")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the first Maxpool layer

In [ ]:
# homemade MaxPool filter on test
out_homemade = maxpool1_homemade(input_batch=out_homemade)
# original MaxPool filter on test
out_original = model_original.maxPool1(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the second Convolutional layer

In [ ]:
# TODO: test the second convolutional layer

# homemade batchNorm1 filter on test
out_homemade = conv2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.conv2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the second Batchnorm layer

In [ ]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the second Maxpool layer

In [ ]:
# homemade batchNorm1 filter on test
out_homemade = maxpool2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.maxPool2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

## Testing the Linear layer

In [ ]:
# homemade linear filter on test
out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(2,45)))
# original linear filter on test
out_original = model_original.lin(torch.reshape(out_original, (2,45)))

print("Linear layer 1 error over out channels: ")
compare(out_homemade.numpy(), list(list(out_original.detach().numpy())))

In [ ]:
import torch.nn as nn

In [ ]:
relu = nn.ReLU()
homemade = ReLU.relu()

print(relu(torch.tensor([0,1,2,-3,-0.32543254,5,6,7])))
print(homemade([[[[0,1,2,-3,0.546547,5,6,7]]]]))